In [0]:
%fs ls dbfs:/FileStore/data-sce/

path,name,size,modificationTime
dbfs:/FileStore/data-sce/company_departments.csv,company_departments.csv,530,1712198093000
dbfs:/FileStore/data-sce/company_regions.csv,company_regions.csv,169,1712198093000
dbfs:/FileStore/data-sce/employees.csv,employees.csv,81451,1712198093000


In [0]:
%sql
SHOW CATALOGS;
CREATE SCHEMA IF NOT EXISTS data_sce;
USE SCHEMA data_sce;

##### Create table employees

In [0]:
%sql
DROP TABLE IF EXISTS employees;
CREATE TABLE employees
  (
    id INT,
    last_name  STRING,
    email  STRING,
    start_date DATE,
    salary INT,
    job_title  STRING,
    region_id INT,
    department_id INT
  )
USING CSV
LOCATION 'dbfs:/FileStore/data-sce/employees.csv';

##### Use of query

In [0]:
%sql
SELECT * FROM  employees
LIMIT 2;

id,last_name,email,start_date,salary,job_title,region_id,department_id
null,kelley,rkelley0@soundcloud.com,null,67470,structural engineer,2,null
null,carr,fcarr2@woothemes.com,null,101768,recruiting manager,3,null


In [0]:
%sql
DESCRIBE FORMATTED data_sce.employees

col_name,data_type,comment
id,int,null
last_name,string,null
email,string,null
start_date,date,null
salary,int,null
job_title,string,null
region_id,int,null
department_id,int,null
,,
# Detailed Table Information,,


##### Create table company_regions

In [0]:
%sql
DROP TABLE IF EXISTS company_regions;
CREATE TABLE company_regions (
	id integer,
	region_name varchar(55),
	country_name varchar(55)
	)
USING csv 
LOCATION 'dbfs:/FileStore/data-sce/company_regions.csv';


##### Use of query

In [0]:
%sql
SELECT * FROM company_regions 
LIMIT 2;

id,region_name,country_name
null,region_name,country_name
1,northeast,usa


In [0]:
%sql
WITH cte AS 
(
  SELECT emp.last_name, emp.email, emp.salary, cr.region_name, cr.country_name
  FROM data_sce.employees emp
  JOIN data_sce.company_regions  cr
  ON emp.region_id = cr.id
)
  SELECT last_name, salary, COUNT(*) FROM cte
  GROUP BY ALL
  HAVING salary > 10000
  LIMIT 2
;

last_name,salary,count(1)
lawson,130993,2
spencer,110881,1


In [0]:
%sql
DROP TABLE IF EXISTS company_departments;
CREATE TABLE company_departments (
	id INT,
    department_name VARCHAR(55),
    division_name VARCHAR(55)
  )
USING CSV
LOCATION 'dbfs:/FileStore/data-sce/company_departments.csv'

##### Use of query

In [0]:
%sql
SELECT * FROM company_departments
LIMIT 2;

id,department_name,division_name
1,automotive,auto & hardware
2,baby,domestic


##### Using join 

In [0]:
%sql
SELECT cr.region_name, cr.country_name, COUNT(*)
FROM employees emp
JOIN company_regions cr
ON emp.region_id = cr.id
GROUP BY cr.region_name, cr.country_name
HAVING LTRIM(cr.country_name) = 'usa';


region_name,country_name,count(1)
northwest,usa,137
northeast,usa,149
southeast,usa,164
southwest,usa,175
